In [1]:
# Prepare all cases
N_h_list = [50, 100] 
noise_list = [1e-2, 5e-2, 1e-1]
dataset_list = ['dis']
method_list = ['lap']
eigen_list = [256] # 32, 64, 128, 256, 512, 1024, 1093


# # Prepare all cases
# N_h_list = [50] 
# noise_list = [1e-2]
# dataset_list = ['hea']
# method_list = ['lap']
# eigen_list = [256] # 32, 64, 128, 256, 512, 1024, 1093

In [ ]:
import autograd.numpy as np
import pandas as pd
import matplotlib.pyplot as plt # for plotting
from sklearn import manifold 
import scipy.io # for loading .mat files
from mpl_toolkits.mplot3d import Axes3D
import time # for timing
import sys # for printing progress+
from scipy.optimize import minimize, fmin_l_bfgs_b # for optimizing
from autograd.numpy.linalg import inv # for matrix inversion
import warnings
warnings.filterwarnings("ignore")

import importlib

# Loop over all cases
for N_h in N_h_list:
    for noise in noise_list:
        for dataset in dataset_list:
            for method in method_list:
                for eigen_num in eigen_list:
                    print(f"\nRunning case: N_h={N_h}, noise={noise}, dataset={dataset}, method={method}, eigen_num={eigen_num}")

                    # Record start time
                    time_start = time.time()

                    # --- Load Data ---
                    import scipy.io as sio

                    if dataset == 'hea':
                        print ("Load healthy data...........")
                        inc = scipy.io.loadmat(r'./Healthy input/heart_coordinates.mat')
                        X = inc['heart_coordinates']
                        X_shape = X.shape[0]

                        t = scipy.io.loadmat(r'./Healthy input/time.mat')
                        t = t['time'].T
                        t_shape = t.shape[0]

                        hsp = scipy.io.loadmat(r'./Healthy input/hsp.mat')
                        hsp = hsp['hsp']
                        hsp_m = hsp.T

                        mat_data = sio.loadmat('./Healthy input/Ven.mat')
                        Tri = mat_data['Ven'][0, 0][1]
                        Tri = Tri - 1
                        Tri = Tri.astype(np.int32)
                        print("Finishing................")

                    elif dataset == 'dis':
                        print ("Load diseased data...........")
                        inc = scipy.io.loadmat(r'./Diseased input/heart_coordinates.mat')
                        X = inc['heart_coordinates']
                        X_shape = X.shape[0]

                        t = scipy.io.loadmat(r'./Diseased input/time_diseased.mat')
                        t = t['time_diseased'].T
                        t_shape = t.shape[0]

                        hsp = scipy.io.loadmat(r'./Diseased input/hsp_diseased.mat')
                        hsp = hsp['hsp_diseased']
                        hsp_m = hsp.T

                        mat_data = sio.loadmat('./Diseased input/Ven.mat')
                        Tri = mat_data['Ven'][0, 0][1]
                        Tri = Tri - 1
                        Tri = Tri.astype(np.int32)
                        print("Finishing................")

                    # --- Loop over all time steps ---
                    total_diff_sq = 0.0
                    total_true_sq = 0.0
                    t_steps = t.shape[0]
                    re_list = []

                    for ti in range(t_steps):
                        # Prepare data for this time step
                        t_i = np.array([t[ti]])
                        hsp_i = hsp[:, ti].reshape(-1, 1)
                        hsp_m_i = hsp_i.T

                        # --- Prepare training and test data ---
                        print(f"\nPrepare for training and test data for time step {ti}...........")

                        Y_noise = hsp_i + noise * np.random.randn(hsp_i.shape[0], hsp_i.shape[1])

                        np.random.seed(1234)
                        idx = np.random.choice(X_shape, N_h, replace=False)
                        idx = np.sort(idx)

                        Y_train_noise = Y_noise[idx,:]
                        print("Y_train_noise_shape:", Y_train_noise.shape)

                        mask = np.ones(hsp_i.shape[0], dtype=bool)
                        mask[idx] = False
                        Y_test_noise = Y_noise[mask,:]
                        print("Y_test_noise_shape:", Y_train_noise.shape)

                        # --- Kernel Calculation ---
                        import Kernel_Time
                        importlib.reload(Kernel_Time)
                        from Kernel_Time import cal_t_kers

                        ker = 1.5 # 1 for Rational Quadratic Kernel ; 0.5 for Matern(1/2); 1.5 for Matern(3/2); 2.5 for Matern(5/2)
                        # For single time point, t_i is 1D or 2D with 1 col
                        func_K_t, func_K_t_s, func_K_t_ss = cal_t_kers(t_i, ker)

                        if method == 'lap':
                            print("Import lap spatial kernel functions..................")
                            print("Solve eigenproblems for all data..................")
                            from Eigen_Lap import eigen_sol
                            Q, V = eigen_sol(X, Tri, num = eigen_num)
                            from Kernel_Space_Lap import cal_sp_kers
                            func_K_sp, func_K_sp_s, func_K_sp_ss = cal_sp_kers(vertex=idx, Q=Q, V=V, kernel_type='Matern', smoothness=3./2.)
                        elif method == 'eu':
                            print("Import euclidean spatial kernel functions..................")
                            from Kernel_Space_Eu import calculate_spatial_kernels_euclidean
                            func_K_sp, func_K_sp_s, func_K_sp_ss = calculate_spatial_kernels_euclidean(idx, X, Tri)

                        # --- Parameter Estimation ---
                        import Pars_MulStarts
                        import Pars_NLL
                        import Eigen_Lap
                        import Block_Diag_Inv

                        importlib.reload(Pars_MulStarts)
                        importlib.reload(Pars_NLL)
                        importlib.reload(Eigen_Lap)
                        importlib.reload(Block_Diag_Inv)

                        print("Begin estimating kernels' parameters.................")
                        from Pars_MulStarts import Pars_Optimizer

                        bounds = ((0.5, 1.5), (800, 900), (0.0005, 0.0005), (50, 60), (1, 1), (0.01, 0.01), (0.0005, 2e-2)) # NOTE: Lap

                        starts = 1
                        np.random.seed(1234)
                        guesses = np.vstack([np.random.uniform(bound[0], bound[1], size=starts) for bound in bounds]).T

                        Optimizer = Pars_Optimizer(
                            Y_train_noise, Y_test_noise, 
                            func_K_t, func_K_t_s, func_K_t_ss,
                            func_K_sp, func_K_sp_s, func_K_sp_ss,
                            X, Tri, mask, t_i
                        ) 

                        best_pars, best_err = None, float('inf')
                        for i, guess in enumerate(guesses):
                            print(f"\nOptimization run {i+1}:")
                            print(f"Initial guess {i+1}: {np.array2string(guess, separator=', ')}")
                            pars = Optimizer.opt_pars(guess, bounds)
                            print(f"Optimized parameters: {np.array2string(pars, separator=', ')}")
                            val_err = 0 # NOTE: for quick testing
                            if val_err < best_err:
                                best_err, best_pars = val_err, pars

                        print("\nBest optimization result:")
                        print(f"Best parameters: {np.array2string(best_pars, separator=', ')}")
                        pars = best_pars

                        pars = np.concatenate([pars, guess[3:]]) # NOTE: construct the full pars

                        # --- Posterior Prediction ---
                        import Post_Pred
                        importlib.reload(Post_Pred)
                        from Post_Pred import Post_Predictor

                        print("Calculate posterior distribution..................")
                        Predictor = Post_Predictor(
                            pars=pars,
                            func_K_t=func_K_t,
                            func_K_t_s=func_K_t_s,
                            func_K_t_ss=func_K_t_ss,
                            func_K_sp=func_K_sp,
                            func_K_sp_s=func_K_sp_s,
                            func_K_sp_ss=func_K_sp_ss
                        )

                        hsp_pred = Predictor.post_mean(Y_train_noise)
                        print("The shape of hsp_pred:", hsp_pred.shape)

                        # --- Visualization Data ---
                        hsp_vis = Y_noise.copy()
                        hsp_vis[mask,:] = hsp_pred.T

                        # --- Error Analysis ---
                        hsp_true = np.array(Y_noise[:, :])
                        hsp_pred_full = np.array(hsp_vis[:, :])

                        diff = hsp_true - hsp_pred_full
                        re_total = np.linalg.norm(diff.reshape(-1)) / np.linalg.norm(hsp_true.reshape(-1))

                        print(f"Time step {ti}: RE = {re_total:.6f}")

                        # Accumulate for total RE
                        total_diff_sq += np.sum(diff**2)
                        total_true_sq += np.sum(hsp_true**2)
                        re_list.append(re_total)

                    # After all time steps
                    if total_true_sq > 0:
                        total_re = np.sqrt(total_diff_sq) / np.sqrt(total_true_sq)
                    else:
                        total_re = np.nan

                    print(f"\nTotal RE over all time steps: {total_re:.6f}")

                    # Record end time and compute elapsed time
                    time_end = time.time()
                    elapsed_time = time_end - time_start
                    print(f"Elapsed time for this case: {elapsed_time:.2f} seconds")

                    # Save result for this case to a separate txt file
                    filename = f"res_Nh{N_h}_noi{noise}_{dataset}_{method}_eig{eigen_num}.txt"
                    with open(filename, 'w') as f:
                        f.write("N_h\tnoise\tdataset\tmethod\teigen_num\tTotal_RE\tpars\telapsed_time_sec\n")
                        f.write(f"{N_h}\t{noise}\t{dataset}\t{method}\t{eigen_num}\t{total_re:.6f}\t{np.array2string(best_pars, separator=', ')}\t{elapsed_time:.2f}\n")
                    print(f"Result for this case saved to {filename}")

print("All cases finished. Each result saved to a separate txt file.")
